In [88]:
import sys
import json
import pandas as pd
import scipy
import sklearn.preprocessing
import numpy as np

In [19]:
als_recommendations = pd.read_parquet("./data/als_recommendations.parquet")

In [26]:
events = pd.read_parquet("./data/events.parquet")

In [20]:
als_recommendations.head()

,user_id,item_id,score
0,1000000,3,0.990942
1,1000000,15881,0.896617
2,1000000,5,0.864405
3,1000000,6,0.822256
4,1000000,2,0.774097


In [21]:
als_recommendations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43058500 entries, 0 to 43058499
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  int64  
 1   item_id  int64  
 2   score    float64
dtypes: float64(1), int64(2)
memory usage: 985.5 MB


In [22]:
top100_recommendations = als_recommendations.sort_values(by='score', ascending=False).head(100)

In [23]:
top100_recommendations.head()

,user_id,item_id,score
5783400,1057834,11387515,2.350093
16444300,1164443,10964,2.234873
13665700,1136657,29056083,2.224305
32614600,1326146,9969571,2.193836
40560200,1405602,10964,2.181578


In [24]:
# расчёт покрытия по объектам
# Предполагаем, что als_recommendations — это DataFrame с колонкой 'item_id', содержащей рекомендации для всех пользователей
# Получаем уникальные объекты, которые присутствуют в рекомендациях
recommended_items = set(top100_recommendations['item_id'].unique())

# Общее количество объектов в системе (все возможные item_id в items)
total_items = len(als_recommendations["item_id"].unique())

# Вычисляем покрытие по объектам
cov_items = len(recommended_items) / total_items

print(f"{cov_items:.2f}")

0.01


In [27]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

Посчитайте среднее Novelty@5 для als_recommendations. Для этого: 
- разметьте каждую рекомендацию в als_recommendations булевым признаком read (False — пользователь не читал книгу, True — пользователь читал книгу), используя events_train,
- посчитайте Novelty@5 для каждого пользователя,
- посчитайте среднеарифметическое для полученных значений Novelty@5.

In [30]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(
    events_train[["user_id", "item_id", "read"]],
    on=["user_id", "item_id"],
    how="left"
)
als_recommendations["read"] = (als_recommendations["read"].fillna(False).astype("bool"))

# проставим ранги
als_recommendations = als_recommendations.sort_values(by=["user_id", "score"], ascending=[True, False])
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
average_novelty_5 = novelty_5.mean()

print(f"{average_novelty_5:.2f}")

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_81636/3355860592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True
/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_81636/3355860592.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")


0.61


Задание 1 из 6

Используем отложенную тестовую часть данных — назовём её events_test — для получения двух новых частей данных:
- одна, составляющая первые 45 дней, будет использоваться для таргетов,
- другая, состоящая из 45 последних дней, будет новой тестовой выборкой.

Завершите код так, чтобы в events_labels оказалась первая часть данных, а в events_test_2 — вторая.

In [31]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

In [33]:
len(events_labels['user_id'].unique())

99849

Задание 2 из 6

Объедините имеющихся кандидатов по совпадению user_id, item_id в один список.

In [35]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("./candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("./candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

In [36]:
len(candidates)

82993094

Задание 3 из 6

Дополните код ниже.
- В candidates добавьте колонку target со значениями:
- 1 для тех item_id, которые пользователь прочитал (положительный пример).
- 0 — для всех остальных (негативный пример).
- В candidates_for_train отберите все положительные примеры, а также не менее четырёх негативных примеров для каждого пользователя в положительных примерах.

In [37]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 
events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]],
                              on=["user_id", "item_id"], how="left")
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
])

/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_81636/1758537609.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(negatives_per_user, random_state=0))


In [38]:
len(candidates_for_train)

213708

Обучение модели

In [39]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6490414	total: 72.5ms	remaining: 1m 12s
100:	learn: 0.5023444	total: 1.32s	remaining: 11.8s
200:	learn: 0.5015587	total: 2.4s	remaining: 9.55s
300:	learn: 0.5008835	total: 3.32s	remaining: 7.71s
400:	learn: 0.5002699	total: 4.23s	remaining: 6.32s
500:	learn: 0.4997442	total: 5.13s	remaining: 5.11s
600:	learn: 0.4992847	total: 6.03s	remaining: 4s
700:	learn: 0.4988372	total: 6.97s	remaining: 2.97s
800:	learn: 0.4984724	total: 7.87s	remaining: 1.95s
900:	learn: 0.4981271	total: 8.77s	remaining: 964ms
999:	learn: 0.4977725	total: 9.68s	remaining: 0us


Для подготовки кандидатов используем новые рекомендации от базовых генераторов, обученных на объединенных данных из events_train и events_label. Эти рекомендации сохранены в файлах als_recommendations.parquet и content_recommendations.parquet в директории candidates/inference. Используем их для формирования списка candidates_to_rank для ранжирующей модели.

Задание 4 из 6

Дополните код ниже так, чтобы в candidates_to_rank попали кандидаты от обоих базовых генераторов подобно тому, как это было сделано для фазы тренировки выше.

In [46]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("./candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("./candidates/inference/content_recommendations.parquet")

In [53]:
candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[
    candidates_to_rank["user_id"]
    .isin(events_test["user_id"].drop_duplicates())
]

In [54]:
print(len(candidates_to_rank))

23830721


Применим обученную ранжирующую модель к кандидатам для рекомендаций. Таргет уже не нужен, поскольку мы будем применять модель в режиме инференса.

Задание 5 из 6

Дополните код для того, чтобы вызвать модель и получить для каждого пользователя топ-100 рекомендаций — значение rank нужно выставить не более ста.

In [49]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

# Добавляем колонку с предсказанным score от модели
candidates_to_rank["cb_score"] = predictions[:, 1]

# Сортируем по пользователю и по убыванию cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

# Проставляем rank, начиная с 1
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

# Оставляем топ-100 рекомендаций для каждого пользователя
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [50]:
print(len(final_recommendations))

12322300


Посчитайте метрики recall и precision.
1. Используйте полученные рекомендации final_recommendations, отложенную тестовую выборку events_test_2, созданные в уроке «Валидация» предыдущей темы.
2. А также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [55]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):
    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")

    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[
        events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)

    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]],
        on=["user_id", "item_id"], how="outer")

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()

    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [56]:
# Функции compute_cls_metrics для расчёта recall.
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [57]:
# Объединяем тренировочные события и метки для валидации
events_inference = pd.concat([events_train, events_labels])

# Обрабатываем рекомендации для бинарных метрик
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5
)

# Вычисляем метрики precision и recall
cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.007, recall: 0.016


/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_81636/2783672476.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events_recs_common["gt"] = events_recs_common["gt"].fillna(False)


In [58]:
items = pd.read_parquet("./data/items.parquet")

In [59]:
# Вычисляем возраст книги и исправляем некорректные значения
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

# Добавляем признаки age и average_rating к кандидатам для тренировки и ранжирования
candidates_for_train = candidates_for_train.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")
candidates_to_rank = candidates_to_rank.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")

In [60]:
# Вычисляем медианный возраст книги для candidates_to_rank
median_age = candidates_to_rank["age"].median()
print(f"Медианный возраст книги: {median_age:.1f}")

Медианный возраст книги: 7.0


Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
- reading_years — длительность истории пользователя,
- books_read — количество книг, прочитанных за всё время,
- books_per_year — среднее количество прочитанных книг в год,
- rating_avg — средняя оценка,
- rating_std — дисперсия оценок.

In [61]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("item_id", "nunique"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std"))
    
    # Заполняем NaN для пользователей с только одной оценкой (std будет NaN)
    user_features["rating_std"] = user_features["rating_std"].fillna(0)
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features

# Получаем признаки пользователей для тренировки
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
  
# Оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

# Получаем признаки пользователей для ранжирования
user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

In [62]:
# Выводим пример данных
print(candidates_for_train.head())
print(candidates_to_rank.head())

   user_id   item_id  als_score  cnt_score  target   age  average_rating  \
0  1000006      7445   0.230529        NaN       1  12.0            4.24   
1  1000006  18812405   0.178382        NaN       1   4.0            3.81   
2  1000006  29868610   0.286715        NaN       1   NaN            3.90   
3  1000019     37415   0.043595        NaN       1  12.0            3.87   
4  1000023      5094   0.082626        NaN       1  15.0            4.23   

   reading_years  books_read  rating_avg  rating_std  books_per_year  
0       1.820671        17.0    4.294118    0.685994        9.337218  
1       1.820671        17.0    4.294118    0.685994        9.337218  
2       1.820671        17.0    4.294118    0.685994        9.337218  
3       0.276523         6.0    4.166667    1.169045       21.698020  
4       0.005476         2.0    3.500000    0.707107      365.250000  
   user_id  item_id  als_score  cnt_score   age  average_rating  \
0  1000003     1232   0.484089        NaN   NaN   

In [63]:
print(candidates_for_train['books_read'].median())

32.0


Задание

Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь. 

Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.

Для экономии ресурсов возьмём не все жанры, а десять наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others. 

In [129]:
def get_genres(items):
    from json import JSONDecodeError
    
    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    genres_counter = {}

    for k, v, in items.iterrows():
        genre_and_votes = v.get('genre_and_votes')
        if genre_and_votes is None or genre_and_votes == 'None':
            continue
        
        try:
            genre_and_votes = json.loads(str(genre_and_votes).replace('\'', '"'))
        except JSONDecodeError as e:
            print(f"JSON decoding error: {e}", genre_and_votes)
            continue
        
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"

    return genres

In [130]:
# Функция в коде ниже строит матрицу вида «книга-жанр».
def get_item2genre_matrix(genres, items):
    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()

    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []

    for item_idx, (k, v) in enumerate(items.iterrows()):
        genre_and_votes = v.get('genre_and_votes')
        if genre_and_votes is None or genre_and_votes == 'None':
            continue
        genre_and_votes = json.loads(str(genre_and_votes).replace('\'','"'))
        for genre_name, votes in genre_and_votes.items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)),
                                         shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)

    return genres_csr

In [131]:
# перекодируем идентификаторы объектов:
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])

In [132]:
genres = get_genres(items)
genres["score"] = genres["votes"] / genres["votes"].sum()

In [140]:
print(genres.head(40))

                                            name    votes         score
genre_id                                                               
0                                        Fantasy  6803715  1.491875e-01
1                                    Young Adult  3281944  7.196435e-02
2                                        Fiction  6393615  1.401951e-01
3                                  Fantasy-Magic   179172  3.928768e-03
4                                      Childrens   439549  9.638147e-03
5                                      Adventure   206550  4.529095e-03
6                                      Audiobook   171304  3.756244e-03
7                         Childrens-Middle Grade   178531  3.914713e-03
8                        Science Fiction Fantasy    64379  1.411661e-03
9                                       Classics  3413828  7.485622e-02
10                                        Novels   110419  2.421197e-03
11                          Fantasy-Supernatural    52436  1.149

In [134]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [135]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    
# Получаем информацию о жанрах для тренировки
user_genres_for_train = get_user_genres(events_train, items, genre_columns)
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

In [138]:
genres.head(20)

,name,votes,score
genre_id,,,
0,Fantasy,6803715,1.491875e-01
1,Young Adult,3281944,7.196435e-02
2,Fiction,6393615,1.401951e-01
3,Fantasy-Magic,179172,3.928768e-03
4,Childrens,439549,9.638147e-03
5,Adventure,206550,4.529095e-03
6,Audiobook,171304,3.756244e-03
7,Childrens-Middle Grade,178531,3.914713e-03
8,Science Fiction Fantasy,64379,1.411661e-03


In [158]:
candidates_to_rank.head()

,user_id,item_id,als_score,cnt_score,age,average_rating,reading_years,books_read,rating_avg,rating_std,...,genre_1,genre_34,genre_17,genre_36,genre_74,genre_14,genre_242,genre_others,cb_score,rank
186,1000003,18143977,0.675044,NaN,4.0,4.31,7.4141,94.0,3.287234,0.712746,...,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763,0.688584,1
190,1000003,18774964,0.363716,NaN,4.0,4.35,7.4141,94.0,3.287234,0.712746,...,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763,0.670243,2
194,1000003,21853621,0.420170,NaN,3.0,4.54,7.4141,94.0,3.287234,0.712746,...,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763,0.656343,3
178,1000003,16158542,0.361447,NaN,5.0,4.33,7.4141,94.0,3.287234,0.712746,...,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763,0.574023,4
150,1000003,10664113,0.486414,NaN,7.0,4.31,7.4141,94.0,3.287234,0.712746,...,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763,0.554248,5


In [141]:
candidates_for_train['genre_34'].median()

0.038488976462249

In [142]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6416190	total: 12.7ms	remaining: 12.7s
100:	learn: 0.4523906	total: 1.54s	remaining: 13.7s
200:	learn: 0.4435134	total: 3.19s	remaining: 12.7s
300:	learn: 0.4371116	total: 4.59s	remaining: 10.7s
400:	learn: 0.4317901	total: 5.59s	remaining: 8.35s
500:	learn: 0.4271651	total: 6.59s	remaining: 6.57s
600:	learn: 0.4230431	total: 7.6s	remaining: 5.05s
700:	learn: 0.4191686	total: 8.63s	remaining: 3.68s
800:	learn: 0.4155816	total: 9.65s	remaining: 2.4s
900:	learn: 0.4122107	total: 10.7s	remaining: 1.17s
999:	learn: 0.4091293	total: 12.4s	remaining: 0us


In [150]:
# Создаем Pool для данных инференса
inference_data = Pool(data=candidates_to_rank[features])

# Получаем вероятности предсказаний для каждого кандидата
predictions = cb_model.predict_proba(inference_data)

# Используем вероятность положительного класса (вероятность того, что рекомендация релевантна)
candidates_to_rank["cb_score"] = predictions[:, 1]

# Сортируем кандидатов по пользователю и по убыванию cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])

# Проставляем rank, начиная с 1
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

# Оставляем топ-100 рекомендаций для каждого пользователя
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [151]:
unique_users_count = final_recommendations["user_id"].nunique()
print(unique_users_count)

123223


In [153]:
final_recommendations.to_parquet("./data/final_recommendations_feat.parquet")

In [159]:
candidates_to_rank.to_parquet("./data/top_recs.parquet")

Итак, вы получили рекомендации, которые уже должны учитывать не только оценки от базовых генераторов als_score и cnt_score, но и информацию, заложенную в признаках. Посмотрим, помогло ли это повысить качество рекомендаций по метрике recall, по которой вы уже оценивали результаты работы модели в прошлом уроке после внедрения двухстадийного подхода. Напомним, что тогда получилось значение 0.016. 

Задание 5 из 6

Используя отложенную тестовую выборку events_test_2, посчитайте метрики recall и precision для полученных рекомендаций.

In [156]:
# Для экономии ресурсов оставляем события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

# Обрабатываем рекомендации для расчёта метрик
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5
)

# Вычисляем метрики precision и recall
cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.013, recall: 0.033


/var/folders/bx/8n6_vdf94z7gqkmk97lf4mnh0000gn/T/ipykernel_81636/2783672476.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events_recs_common["gt"] = events_recs_common["gt"].fillna(False)


Выполните код для получения информации о важности признаков. Выведите список признаков feature_importance в порядке убывания их важности.

In [157]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])

# Сортируем признаки по убыванию их важности
feature_importance = feature_importance.sort_values(by="fi", ascending=False)

print(feature_importance)

                       fi
als_score       27.966070
age             23.154312
average_rating  17.765676
books_read       3.171369
cnt_score        2.520950
reading_years    2.369903
genre_1          2.313234
genre_2          2.258605
genre_others     2.257026
genre_34         2.043743
genre_0          1.960147
genre_9          1.752833
books_per_year   1.657658
genre_14         1.444221
genre_242        1.434225
rating_avg       1.347180
genre_74         1.270173
genre_36         1.170937
genre_17         1.124080
rating_std       1.017657
